In [ ]:
!pip install -q transformers
!pip install -q evaluate
!pip install -q datasets
!pip install -q pandas
#!pip install -q torch
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.

In [ ]:
import evaluate
from pprint import pprint
import pandas as pd
from datasets import load_dataset
# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
#from tqdm import tqdm
#from sentence_transformers import SentenceTransformer, util
#import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# define functions
# using code from here: https://medium.com/@ajazturki10/simplifying-language-understanding-a-beginners-guide-to-question-answering-with-t5-and-pytorch-253e0d6aac54
def prepare_data(data):
  articles = []

  for paragraph in data:
    context = paragraph['context']
    for qa in paragraph['qas']:
      question = qa['question']
      id = qa['id']
      for ans in qa['answers']:
        answer = ans['text']
        answer_start = ans['answer_start']
        articles.append({'context': context, 'question': question, 'id': id, 'answer': answer, 'answer_start': answer_start})

  return articles

def evaluate_abstractive(result_df,
                         pred_col,
                         ref_col='answer',
                         encoder_model='sentence-transformers/all-MiniLM-L12-v2'):
    predictions = result_df[pred_col].tolist()
    references = result_df[ref_col].tolist()

    rouge = evaluate.load('rouge')
    rouge_res = rouge.compute(predictions=predictions,
                              references=references)

    encoder_model = SentenceTransformer(encoder_model)
    candidate_embeddings = encoder_model.encode(predictions)
    reference_embeddings = encoder_model.encode(references)
    similarity = util.pairwise_cos_sim(candidate_embeddings, reference_embeddings)

    print('rouge scores:')
    pprint(rouge_res)
    print()
    print('average semantic similarity:')
    print(torch.mean(similarity))

In [ ]:
ds = load_dataset("rony/climate-change-MRC")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/633 [00:00<?, ?B/s]

CCMRC_train.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

CCMRC_validation.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

CCMRC_test.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
# score T5 on question answering on validation & test datsets
# train_ds = ds["train"]
# valid_ds = ds["validation"]
test_ds = ds["test"]

# each is a 1-item list, so take first index
# train_ds = train_ds[0]
# valid_ds = valid_ds[0]
test_ds = test_ds[0]

# take the 'data' key of the dict, ignoring 'version' (there's just one)
# train_ds = train_ds['data'][0]['paragraphs']
# valid_ds = valid_ds['data'][0]['paragraphs']
test_ds = test_ds['data'][0]['paragraphs']
# each dataset is a list of dicts, where each list item is a context paragraph ('context' key) with qas ('qas' key) which contain questions, id, and answer

# train_df = pd.DataFrame(prepare_data(train_ds))
# print(f"{train_df.shape=}")

# valid_df = pd.DataFrame(prepare_data(valid_ds))
# print(f"{valid_df.shape=}")

test_df = pd.DataFrame(prepare_data(test_ds))
print(f"{test_df.shape=}")

test_df.shape=(2096, 5)


In [ ]:
del ds

In [ ]:
# load saved small saved model
from transformers import T5ForConditionalGeneration, T5Tokenizer

trained_t5small_model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/mids-w266/final_project/t5small')
t5_tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/mids-w266/final_project/t5small')

In [ ]:
test_input_texts = []
for _, row in test_df[:100].iterrows():
  input_text = f"question: {row['question']}  context: {row['context']}"
  test_input_texts.append(input_text)
test_inputs = t5_tokenizer(test_input_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
#predicted_outputs = trained_t5small_model.generate(test_inputs['input_ids'])
rouge = evaluate.load('rouge')

decoded_preds = t5_tokenizer.batch_decode(predicted_outputs, skip_special_tokens=True)
decoded_labels = test_df['answer'][:100].tolist()

rouge_results = rouge.compute(predictions=decoded_preds, references=decoded_labels)
print(rouge_results)

{'rouge1': np.float64(0.5063238278306301), 'rouge2': np.float64(0.446007967168349), 'rougeL': np.float64(0.49540694035311966), 'rougeLsum': np.float64(0.4976051992954991)}


In [16]:
def predict_test(test_df, batch_size=20):
  all_predictions = []
  start = 0
  end = batch_size

  for _ in range(test_df.shape[0]//batch_size + 1):
    test_batch = test_df[start:end]

    test_input_texts = []

    for _, row in test_batch.iterrows():
      input_text = f"question: {row['question']}  context: {row['context']}"
      test_input_texts.append(input_text)
    test_inputs = t5_tokenizer(test_input_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    predictions = trained_t5small_model.generate(test_inputs['input_ids'])
    decoded_predictions = t5_tokenizer.batch_decode(predictions, skip_special_tokens=True)

    all_predictions.append(decoded_predictions)

    start += batch_size
    end += batch_size

  return all_predictions

In [17]:
all_predictions = predict_test(test_df)

In [21]:
predictions = []

for batch in all_predictions:
  for pred in batch:
      predictions.append(pred)

len(predictions)

2096

In [23]:
rouge = evaluate.load("rouge")


all_labels = test_df['answer'].tolist()

rouge_results = rouge.compute(predictions=predictions, references=all_labels)
print(rouge_results)

{'rouge1': np.float64(0.5704078875477627), 'rouge2': np.float64(0.5268776428029627), 'rougeL': np.float64(0.5609993603484609), 'rougeLsum': np.float64(0.5612683578070725)}
